In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Please upvote this notebook if you find it useful!**

# **Hey Guys!**

Here we are going to learn about how to choose best classifier from **Random Forest,Decision Tree,Naive Bias,Support Vector Classification,Logistic Regression,K Neighbours Classifier and SGD Classifier** for our model on wine quality dataset with best and optimal hyperparameters.

**So let's start and get ready to grab new things.**

# **Let' take an overview of this notebook-**

1.Data Visualisation<br/>
2.Splitting the dataset<br/>
3.SVM<br/>
4.Random Forest<br/>
5.Logistic Regression<br/>
6.Guassian Naive Bias<br/>
7.SGD Classifier<br/>
8.KNN Classification<br/>
9.Decision Tree<br/>
10.Hyperparameter tuning using Grid Search CV<br/>
11.Classification Report<br/>

## Importing required libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## Overview of Dataset

In [ ]:
df=pd.read_csv('/kaggle/input/wine-quality-dataset/WineQT.csv')
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

#### So, there is no null value and all the variables are in the numerical form.

In [ ]:
df.describe()

### Finding the number distinct values in variables

In [ ]:
for i, column in enumerate(df.columns):
    print("{}. ".format(i) + str(column.title()) + ": {}". format(df[column].nunique()))

### Plotting the bar graph for target variable 'Quality' and response variable

In [ ]:
col=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 
     'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol', 'Id']
for i in col:
    fig = plt.figure(figsize = (6,4))
    sns.barplot(x = 'quality', y = i, data = df)

#### From the above bar graph, we find that there is minimal change in quality w.r.t fixed acidity,residual sugar,density and ph.

#### Making binary classification for response variable with values between 2 to 6.5 as good and 6.5 to 8 as bad quality using pandas bins and cut method.

In [ ]:
bins = (2, 6.5, 8) 
group_names = ['bad', 'good']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

In [ ]:
df['quality'].value_counts()

### Here we can see that the quality variable is divided into good and bad based on the given limit.

In [ ]:
df.head(5)

### Seperating the dataset into target and response variable

In [ ]:
X = df.drop('quality', axis = 1)
y = df['quality']

###  Splitting the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Applying Standard scaling as some of the models gives best result with this feature.

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

### Now we train our dataset with different models.
### So before moving forward lets learn about these models.

All these comes under supervised machine learning algorithm.</br>
1.Support Vector Classification - A SVM is a very powerful and versatile ML model capable of performing linear and non linear classification,regression and even outlier detection.These are sensitive to feature scaling.The goal of the SVM is to create hyperplane that can segregate n-dimensional space into classes so that we can easily put the new data point in the correct category in the future.SVM chooses the extreme points/vectors that help in creating the hyperplane. These extreme cases are called as support vectors, and hence algorithm is termed as Support Vector Machine.SVM can be of two types linear and non-linear.The hyperplane with maximum margin is called the optimal hyperplane.

2.Random Forest - Random Forest is a classifier that contains a number of decision trees on various subsets of the given dataset and takes the average to improve the predictive accuracy of that dataset. It is based on the concept of ensemble learning, which is a process of combining multiple classifiers to solve a complex problem and to improve the performance of the model.The greater number of trees in the forest leads to higher accuracy and prevents the problem of overfitting.It takes less training time as compared to other algorithms.It can also maintain accuracy when a large proportion of data is missing.

3.Logistic Regression - Logistic regression uses the concept of predictive modeling as regression; therefore, it is called logistic regression, but is used to classify samples; Therefore, it falls under the classification algorithm.It predicts the output of a categorical dependent variable. Therefore the outcome must be a categorical or discrete value. It can be either Yes or No, 0 or 1, true or False, etc. but instead of giving the exact value as 0 and 1, it gives the probabilistic values which lie between 0 and 1.It uses sigmoid function to classify the data.

4.Gaussian Naive Bias - It is based on Bayes theorem and used for solving classification problems.It is a probabilistic classifier, which means it predicts on the basis of the probability of an object.The Gaussian model assumes that features follow a normal distribution. This means if predictors take continuous values instead of discrete, then the model assumes that these values are sampled from the Gaussian distribution.

5.SGD Classifier-The word ‘stochastic‘ means a system or a process that is linked with a random probability. Hence, in Stochastic Gradient Descent, a few samples are selected randomly called batch instead of the whole data set for each iteration.It basically implements a plain SGD learning routine supporting various loss functions and penalties for classification.

6.KNN Classifier - KNN algorithm assumes the similarity between the new case/data and available cases and put the new case into the category that is most similar to the available categories.It stores all the available data and classifies a new data point based on the similarity. This means when new data appears then it can be easily classified into a well suite category by using this algorithm.It is also called a lazy learner algorithm because it does not learn from the training set immediately instead it stores the dataset and at the time of classification, it performs an action on the dataset.Eucledian distance is one of the methods used to calculate nearest neighbours.

7.Decision Tree Classifier-It is a tree-structured classifier, where internal nodes represent the features of a dataset, branches represent the decision rules and each leaf node represents the outcome.A decision tree simply asks a question, and based on the answer (Yes/No), it further split the tree into subtrees.The decision tree contains lots of layers, which makes it complex.It may have an overfitting issue, which can be resolved using the Random Forest algorithm.

### Analysis of classification report for each model

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
pred_svc = svc.predict(X_test)
print(classification_report(y_test, pred_svc))

#### Support vector classifier gets an accuracy of 87%.

In [ ]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
print(classification_report(y_test, pred_rfc))

#### Random Forest classifier gets an accuracy of 89%.

In [ ]:
sgd = SGDClassifier(penalty=None)
sgd.fit(X_train, y_train)
pred_sgd = sgd.predict(X_test)
print(classification_report(y_test, pred_sgd))

#### SGD classifier gets an accuracy of 86%.

In [ ]:
log = LogisticRegression()
log.fit(X_train, y_train)
pred_log = log.predict(X_test)
print(classification_report(y_test, pred_log))

#### Logistic Regression gets an accuracy of 89%.

In [ ]:
NB = GaussianNB()
NB.fit(X_train, y_train)
pred_NB = NB.predict(X_test)
print(classification_report(y_test, pred_NB))

#### Guassian Naive Bias classifier gets an accuracy of 89%.

In [ ]:
knn=KNeighborsClassifier()
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)
print(classification_report(y_test, pred_knn))

#### KNN classifier gets an accuracy of 86%.

In [ ]:
dtc= DecisionTreeClassifier()
dtc.fit(X_train, y_train)
pred_dtc = dtc.predict(X_test)
print(classification_report(y_test, pred_dtc))

#### Decision Tree classifier gets an accuracy of 85%.

### Grid Search CV

GridSearchCV identifies the best technically parameters and models, but only that are included in parameter grid. We do this by defining a dictionary in which we mention a particular hyperparameter along with the values it can take. 

In cross-validation(CV), the process divides the train data further into two parts – the train data and the validation data.

**Here we observe that after applying Grid search CV our best scores increases.**

In [ ]:
model_params = {
    'SVM': {
        'model': SVC(gamma='auto'),
        'params' : {
                'C': [0.1,0.8,0.9,1,1.1,1.2,1.3,1.4],
                'kernel':['linear', 'rbf' ,'poly'],
                'gamma' :[0.1,0.8,0.9,1,1.1,1.2,1.3,1.4]
        }  
    },
    'Random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10,20,80,100,140,160,180,200]
        }
    },
    'Logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [0.1,0.5,0.8,1,5]
        }
    },
    'Naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'SGD_classifier': {
        'model': SGDClassifier(),
        'params': {}
    },
    'KNN_classifier':{
        'model': KNeighborsClassifier(),
        'params' : {}
    },
    'Decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
            
        }
    }     
}

### This is the best scores and best parameters for all the models.

In [ ]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=10, return_train_score=False)
    clf.fit(X_train,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
dfg = pd.DataFrame(scores,columns=['model','best_score','best_params'])
dfg

In [ ]:
#Let's run our SVC again with the best parameters.
svc2 = SVC(C = 1.2, gamma =  0.8, kernel= 'rbf')
svc2.fit(X_train, y_train)
pred_svc2 = svc2.predict(X_test)
print(classification_report(y_test, pred_svc2))

### Here we observe that the accuracy of SVC model increases from 87% to 94% using Grid Search CV.

### Thus, Grid Search CV is a useful technique to identify best and optimal parameters.

## If you find this notebook helpful, then please upvote as it can be useful for others also.